In [23]:
import sys
import mlflow
import pandas as pd
from mlflow.tracking import MlflowClient
sys.path.append('../src/')
from helpers.mlflow_utils import mlflow_tracking_uri
import matplotlib.pyplot as plt

client = MlflowClient(tracking_uri=mlflow_tracking_uri)
experiment_id = '206'
unlearning_methods = ['finetune', 'neggrad', 'relabel', 'badT', 'scrub', 'ssd', 'unsir', 'our']
runs = client.search_runs(experiment_id)
# Convert runs to a DataFrame
metrics = ["mia", "acc_forget", "acc_retain", "t", "acc_test", "js", "js_proxy"]
runs_df = pd.DataFrame([{k: v for k, v in run.data.metrics.items() if k in metrics} for run in runs])
runs_df['method'] = [run.data.tags.get('mlflow.runName') for run in runs]
runs_df['seed'] = [run.data.params.get('seed') for run in runs]
runs_df = runs_df.set_index(['method', 'seed'])

gap_metrics = ['mia', 'acc_forget', 'acc_retain', 'acc_test']
unlearning_methods = ['finetune', 'neggrad', 'relabel', 'badT', 'scrub', 'ssd', 'unsir', 'our', 'retrained']
# Calculate the difference in 't' between the unlearning methods and 'retrain' of the same seed
for method in unlearning_methods:
    for metric in gap_metrics:
        for seed in runs_df.index.get_level_values('seed').unique():
            runs_df.loc[method, f'{metric}_gap'] = abs(runs_df.loc[(method, seed), metric] - runs_df.loc[('retrained', seed), metric])

grouped_df = runs_df.groupby('method').aggregate(['mean', 'std'])
grouped_df['js'] = grouped_df['js'].apply(lambda x: x*1e6)
grouped_df = grouped_df.round(2)

# Display the updated DataFrame
grouped_df


acc_retain       acc_test         mia           t            js  \
                mean   std     mean   std  mean   std  mean   std    mean   
method                                                                      
badT            0.71  0.18     0.66  0.16  0.43  0.37  0.33  0.00  238.67   
finetune        0.98  0.01     0.89  0.02  0.30  0.26  0.43  0.00  103.33   
neggrad         0.97  0.01     0.88  0.02  0.48  0.50  0.49  0.00  105.67   
original        1.00  0.00     0.93  0.01  0.94  0.00  6.50  0.13     NaN   
our             0.85  0.07     0.77  0.07  0.67  0.58  0.29  0.00  147.33   
relabel         0.97  0.01     0.89  0.02  0.48  0.50  0.57  0.00  100.33   
retrained       0.99  0.02     0.91  0.02  0.76  0.03  5.32  1.18    0.00   
scrub           1.00  0.00     0.93  0.01  0.94  0.01  0.58  0.00   41.33   
ssd             0.87  0.16     0.80  0.15  0.81  0.11  0.54  0.00   82.33   
unsir           0.93  0.01     0.86  0.01  0.46  0.03  0.45  0.00   65.00   

                   ... acc_forget       mia_gap      acc_forget_gap       \
              std  ...       mean   std    mean  std           mean  std   
method             ...                                                     
badT       203.46  ...       0.71  0.18    0.73  0.0           0.37  0.0   
finetune    24.21  ...       0.97  0.01    0.27  0.0           0.08  0.0   
neggrad     21.01  ...       0.97  0.01    0.29  0.0           0.08  0.0   
original      NaN  ...       1.00  0.00     NaN  NaN            NaN  NaN   
our         50.65  ...       0.78  0.07    0.73  0.0           0.04  0.0   
relabel     25.54  ...       0.96  0.01    0.27  0.0           0.06  0.0   
retrained    0.00  ...       0.91  0.02    0.00  0.0           0.00  0.0   
scrub        9.24  ...       1.00  0.00    0.20  0.0           0.11  0.0   
ssd         57.27  ...       0.86  0.16    0.09  0.0           0.03  0.0   
unsir        4.58  ...       0.93  0.01    0.25  0.0           0.03  0.0   

          acc_retain_gap      acc_test_gap       
                    mean  std         mean  std  
method                                           
badT                0.44  0.0         0.40  0.0  
finetune            0.01  0.0         0.01  0.0  
neggrad             0.01  0.0         0.01  0.0  
original             NaN  NaN          NaN  NaN  
our                 0.03  0.0         0.04  0.0  
relabel             0.01  0.0         0.01  0.0  
retrained           0.00  0.0         0.00  0.0  
scrub               0.04  0.0         0.03  0.0  
ssd                 0.03  0.0         0.04  0.0  
unsir               0.04  0.0         0.04  0.0  

[10 rows x 22 columns]

In [ ]:
# # Calculate average and standard deviation per seed
# grouped_df = runs_df.groupby('method')[metrics].agg(['mean', 'std'])
# for method in unlearning_methods:
#     for metric in ["mia", "acc_forget", "acc_retain", "acc_test"]:
#         grouped_df.loc[method, (metric, 'gap')] = abs(grouped_df.loc[method, (metric, 'mean')] - grouped_df.loc['retrained', (metric, 'mean')])
# grouped_df = grouped_df.sort_index(axis=1, level=0)
# grouped_df['avg_gap'] = grouped_df[[col for col in grouped_df.columns if col[1] == 'gap']].mean(axis=1)

# # Format the values to have only 2 decimals, apart from js and js_proxy that would have 6
# for col in grouped_df.columns:
#     if col[0] in ['js', 'js_proxy']:
#         grouped_df[col] = grouped_df[col].apply(lambda x: f"{x:.6f}")
#     else:
#         grouped_df[col] = grouped_df[col].apply(lambda x: f"{x:.2f}")

# grouped_df

# time_means = grouped_df[('t', 'mean')].astype(float)
# time_stds = grouped_df[('t', 'std')].astype(float)
# time_means = time_means.drop(['original', 'retrained'])
# time_stds = time_stds.drop(['original', 'retrained'])

# # Sort the means and standard deviations
# time_means_sorted = time_means.sort_values()
# time_stds_sorted = time_stds[time_means_sorted.index]

# plt.figure(figsize=(6, 4))
# plt.bar(time_means_sorted.index, time_means_sorted, yerr=time_stds_sorted, capsize=5, color='skyblue')
# plt.title('Execution Time per Method')
# plt.xlabel('Method')
# plt.ylabel('Execution Time (min)')
# plt.xticks(rotation=45)
# plt.show()